In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

train.csv



In [3]:
train = pd.read_csv('./input/train.csv')
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
import nltk as nl

In [5]:
train['tokens'] = [nl.word_tokenize(sentences) for sentences in train.text]

In [6]:
words = []
for item in train.tokens:
    words.extend(item)

In [7]:
stemmer = nl.stem.lancaster.LancasterStemmer()

In [8]:
words = [stemmer.stem(word) for word in words]
words = set(words)

In [9]:
training = []
for index,item in train.iterrows():
    onehot = []
    token_words = [stemmer.stem(word) for word in item['tokens']]
    for w in words:
        onehot.append(1) if w in token_words else onehot.append(0)
    
    training.append([onehot,item['author']])

In [10]:
training_new = np.array(training)

In [11]:
import sklearn.preprocessing
training_new[:,1].shape
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(4))
train_y = label_binarizer.transform(training_new[:,1])

In [14]:
training_new[:,1]

array(['EAP', 'HPL', 'EAP', ..., 'EAP', 'EAP', 'HPL'], dtype=object)

In [19]:


from numpy import array

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(training_new[:,1])

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)




array([ 1.,  0.,  0.])

In [54]:
onehot_encoded[1]

array([ 0.,  1.,  0.])

In [55]:
# invert first example

inverted = label_encoder.inverse_transform([argmax(onehot_encoded[1, :])])
print(inverted)

['HPL']


In [20]:
train_x = list(training_new[:,0])
train_y = onehot_encoded

In [22]:
import tensorflow as tf
import tflearn
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
 
# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=10, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 24479  | total loss: 0.08751 | time: 10.211s
| Adam | epoch: 010 | loss: 0.08751 - acc: 0.9819 -- iter: 19576/19579
Training Step: 24480  | total loss: 0.08110 | time: 10.214s
| Adam | epoch: 010 | loss: 0.08110 - acc: 0.9837 -- iter: 19579/19579
--
INFO:tensorflow:/Users/dmurugan/mess/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [23]:
test = pd.read_csv('./input/test.csv')
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [24]:
test['tokens'] = [nl.word_tokenize(sentences) for sentences in test.text]

In [25]:
testing = []
for index,item in test.iterrows():
    onehot = []
    token_words = [stemmer.stem(word) for word in item['tokens']]
    for w in words:
        onehot.append(1) if w in token_words else onehot.append(0)
    
    testing.append(onehot)

In [26]:
testing = list(np.array(testing))

In [27]:
predicted = model.predict(X=testing)

In [30]:
from math import floor
def roundDown(n, d=3):
    d = int('1' + ('0' * d)) 
    return (floor(n * d) / d) * 10

predicted_new = np.vectorize(roundDown)(predicted)

In [32]:
predicted_new[0]

array([ 0.13,  0.  ,  9.86])

In [34]:
result = pd.DataFrame(columns=['id'])
result['id'] = test['id']

In [39]:
test.count()

id        8392
text      8392
tokens    8392
dtype: int64

In [60]:
val_v1 = pd.DataFrame(predicted_new)

In [58]:
result.count()

id    8392
dtype: int64

In [88]:
result_val =pd.DataFrame(predicted)

In [89]:
result_val.columns = ["EAP","HPL","MWS"]

In [90]:
result['EAP'] = result_val['EAP']
result['HPL'] = result_val['HPL']
result['MWS'] = result_val['MWS']

In [94]:
result.head()

,id,EAP,HPL,MWS
0,id02310,0.013221,4.997771e-07,9.867787e-01
1,id24541,0.001353,9.986457e-01,1.570390e-06
2,id00134,0.893018,1.024197e-01,4.562271e-03
3,id27757,0.999999,4.641427e-08,7.497516e-07
4,id04081,0.991057,3.213839e-04,8.621737e-03


In [93]:
result.to_csv("my_submission_v1.csv", encoding='utf-8',index=False)